In [17]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import time


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
solve_season = '2024-25'
solve_gameweek = 14
load_projections_from_file = False

In [19]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

if not load_projections_from_file:
    point_projections = generate_projections()
    stat_projections = generate_stat_projections()

    projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)
    projections_data.to_csv('gameweek_projections.csv', index=False)
    print('Generated new projections.')
else:
    try:
        projections_data = pd.read_csv('gameweek_projections.csv')
        print('Loaded projections from CSV file.')
    except FileNotFoundError:
        print('Tried to load from CSV file, but it does not exist. Generating new projections...')
        point_projections = generate_projections()
        stat_projections = generate_stat_projections()
        projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)

Generated new projections.


### Player Manipulation

### Player Force/Banning

In [20]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


In [21]:
def fuzzy_force_players(df, force_ids):
    while True:
        search_name = input("Enter player name to force (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to force (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    force_ids.append(selected_index)  # Add to force_ids instead of ban_ids
                    print(f"Forced: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return force_ids

force_ids = []
force_ids = fuzzy_force_players(projections_data, force_ids)
print("Final force list (indices):", force_ids)

Final force list (indices): []


# 2024/25 GW14 Challenge: Red Hot Deal - Man United players earn double points

In [22]:
# Double points for Man United players
double_points_teams = ['Man Utd']
projections_data.loc[projections_data['Team'].isin(double_points_teams), 'Predicted_Points'] *= 2

### Optimisation

In [23]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 11
model += plp.lpSum(lineup) == 11

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

for id in force_ids:
    model += lineup[id] == 1

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# Maximum 5 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) <= 5

# Maximum 5 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) <= 5

# Exactly 1 Forward
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) == 1

# Budget constraint
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) <= 75

# Maximum 5 players from Man Utd
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Team'] == 'Man Utd']) <= 5

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

def solution_to_txt(players_dict, filename="solution.txt", encoding="utf-8"):
    total_points = 0
    total_cost = 0
    with open(filename, 'w', encoding=encoding) as f:
        f.write(f'Current Date & Time: {time.strftime("%Y-%m-%d")} - {time.strftime("%H:%M:%S")}\n')
        for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            if position in players_dict:
                f.write(f"\n{position}:\n")
                for player in players_dict[position]:
                    captain_str = " (C)" if player['Captain'] else ""
                    points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                    f.write(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}\n")
                    total_points += points
                    total_cost += player['Cost']
        f.write(f"\nTotal Predicted Points: {round(total_points, 2)}\n")
        f.write(f"Total Cost: {round(total_cost, 2)}m\n")
        
solution_to_txt(selected_players, "optimal_solution.txt")
print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Onana - Man Utd - Cost: 5.2m - Predicted Points: 6.38

Defender:
  Cash - Aston Villa - Cost: 4.4m - Predicted Points: 4.14
  Alexander-Arnold - Liverpool - Cost: 6.9m - Predicted Points: 4.75
  Walker - Man City - Cost: 5.2m - Predicted Points: 4.29
  Dalot - Man Utd - Cost: 5.2m - Predicted Points: 5.06

Midfielder:
  Saka - Arsenal - Cost: 10.4m - Predicted Points: 7.1
  Palmer - Chelsea - Cost: 11.0m - Predicted Points: 7.68
  Amad - Man Utd - Cost: 5.0m - Predicted Points: 7.28
  B.Fernandes (C) - Man Utd - Cost: 8.5m - Predicted Points: 19.36
  Casemiro - Man Utd - Cost: 4.8m - Predicted Points: 6.66

Forward:
  N.Jackson - Chelsea - Cost: 8.1m - Predicted Points: 5.85

Total Predicted Points: 78.55
Total Cost: 74.7m


In [25]:
def print_top_scorers_by_position(projections_data):
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        top_scorers = projections_data[projections_data['Position'] == position].nlargest(10, 'Predicted_Points')
        print(f'Top {position}s: ')
        display(top_scorers)

if print_solution:
    print_top_scorers_by_position(projections_data)

Top Goalkeepers: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
224,383,Onana,Man Utd,38,Goalkeeper,5.2,6.38,90,Arsenal (A),0.0,0.0,0.0,0.125,0.0
11,15,Raya,Arsenal,200,Goalkeeper,5.5,4.30,90,Man Utd (H),0.0,0.0,0.0,0.454,0.0
129,235,Pickford,Everton,241,Goalkeeper,4.9,4.29,90,Wolves (H),0.0,0.0,0.0,0.333,0.0
329,568,Muric,Ipswich,286,Goalkeeper,4.4,4.00,90,Crystal Palace (H),0.0,0.0,0.0,0.267,0.0
105,185,Sánchez,Chelsea,200,Goalkeeper,4.8,3.83,90,Southampton (A),0.0,0.0,0.0,0.444,0.0
138,248,Leno,Fulham,80,Goalkeeper,5.0,3.71,90,Brighton (H),0.0,0.0,0.0,0.267,0.0
162,293,Hermansen,Leicester,58,Goalkeeper,4.5,3.63,90,West Ham (H),0.0,0.0,0.0,0.250,0.0
202,347,Ederson M.,Man City,30,Goalkeeper,5.5,3.58,68,Nott'm Forest (H),0.0,0.0,0.0,0.476,0.0
86,146,Verbruggen,Brighton,152,Goalkeeper,4.5,3.50,90,Fulham (A),0.0,0.0,0.0,0.222,0.0
283,488,Forster,Spurs,241,Goalkeeper,4.3,3.50,90,Bournemouth (A),0.0,0.0,0.0,0.200,0.0


Top Defenders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
217,369,Dalot,Man Utd,173,Defender,5.2,5.06,90,Arsenal (A),0.051,0.057,0.105,0.125,0.059
338,593,De Ligt,Man Utd,152,Defender,5.0,4.92,90,Arsenal (A),0.049,0.045,0.091,0.125,0.056
175,311,Alexander-Arnold,Liverpool,241,Defender,6.9,4.75,85,Newcastle (A),0.087,0.200,0.269,0.325,0.097
339,594,Mazraoui,Man Utd,145,Defender,4.7,4.70,82,Arsenal (A),0.047,0.039,0.084,0.125,0.054
192,335,Robertson,Liverpool,243,Defender,5.9,4.58,78,Newcastle (A),0.072,0.170,0.229,0.333,0.081
213,363,Walker,Man City,241,Defender,5.2,4.29,84,Nott'm Forest (H),0.065,0.098,0.157,0.476,0.075
334,578,Calafiori,Arsenal,106,Defender,5.8,4.26,90,Man Utd (H),0.078,0.107,0.176,0.437,0.088
204,350,Gvardiol,Man City,97,Defender,6.2,4.24,68,Nott'm Forest (H),0.110,0.110,0.208,0.476,0.122
25,32,Cash,Aston Villa,172,Defender,4.4,4.14,79,Brentford (H),0.128,0.083,0.201,0.312,0.149
211,361,Rúben,Man City,173,Defender,5.4,4.10,79,Nott'm Forest (H),0.091,0.040,0.127,0.476,0.103


Top Midfielders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
215,366,B.Fernandes,Man Utd,173,Midfielder,8.5,9.68,90,Arsenal (A),0.132,0.132,0.246,0.125,0.150
103,182,Palmer,Chelsea,241,Midfielder,11.0,7.68,90,Southampton (A),0.490,0.347,0.667,0.444,0.714
214,364,Amad,Man Utd,54,Midfielder,5.0,7.28,72,Arsenal (A),0.123,0.102,0.213,0.125,0.138
13,17,Saka,Arsenal,241,Midfielder,10.4,7.10,86,Man Utd (H),0.360,0.300,0.552,0.454,0.482
187,328,M.Salah,Liverpool,63,Midfielder,13.2,7.10,90,Newcastle (A),0.430,0.297,0.599,0.333,0.592
216,368,Casemiro,Man Utd,30,Midfielder,4.8,6.66,90,Arsenal (A),0.065,0.077,0.137,0.119,0.073
225,385,Rashford,Man Utd,241,Midfielder,6.9,6.64,71,Arsenal (A),0.139,0.076,0.204,0.117,0.161
290,503,Son,Spurs,114,Midfielder,9.9,5.75,81,Bournemouth (A),0.346,0.200,0.477,0.200,0.463
218,372,Garnacho,Man Utd,10,Midfielder,6.4,5.58,58,Arsenal (A),0.000,0.000,0.000,0.000,0.000
203,348,Foden,Man City,241,Midfielder,9.1,5.47,82,Nott'm Forest (H),0.363,0.218,0.502,0.476,0.481


Top Forwards: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
205,351,Haaland,Man City,161,Forward,15.0,7.29,90,Nott'm Forest (H),0.573,0.141,0.633,0.476,0.902
219,375,Højlund,Man Utd,58,Forward,6.9,6.20,69,Arsenal (A),0.000,0.000,0.000,0.000,0.000
101,180,N.Jackson,Chelsea,189,Forward,8.1,5.85,87,Southampton (A),0.385,0.158,0.482,0.433,0.531
41,58,Watkins,Aston Villa,241,Forward,9.0,5.82,84,Brentford (H),0.407,0.171,0.509,0.321,0.576
2,4,Havertz,Arsenal,80,Forward,7.9,5.47,90,Man Utd (H),0.348,0.167,0.457,0.454,0.451
312,541,Cunha,Wolves,30,Forward,7.1,5.29,90,Everton (A),0.273,0.174,0.399,0.250,0.333
113,207,Mateta,Crystal Palace,73,Forward,7.2,5.17,90,Ipswich (A),0.283,0.125,0.373,0.304,0.353
140,252,Raúl,Fulham,139,Forward,5.6,5.13,77,Brighton (H),0.311,0.143,0.409,0.257,0.399
260,447,Wood,Nott'm Forest,154,Forward,6.6,5.12,85,Man City (A),0.189,0.053,0.232,0.111,0.223
68,110,Wissa,Brentford,50,Forward,6.2,4.84,83,Aston Villa (A),0.200,0.096,0.277,0.150,0.242
